In [ ]:
import sys
import warnings
import pandas as pd

sys.path.append('..')
from pytextclassifier import BertClassifier

from loguru import logger

# 移除 debug 日志
logger.remove()
logger.add(sys.stderr, level="WARNING")

In [ ]:
num_epochs = 2
lr = 3e-5
max_seq_length = 512

input_model_name = 'models/bert-base-chinese'
model_name = f'models/train_5w_all_data0723_ip_name_{num_epochs}epoch_seq{max_seq_length}_{lr}'

model_name
m = BertClassifier(output_dir=model_name, num_classes=3,
                   model_type='bert', model_name='models/bert-base-chinese')

m.load_model()

In [ ]:
eval_df = pd.read_csv('./data/val_2w_all_data0723_without_ip.csv')
eval_df = eval_df[eval_df['final_text'].notnull()]
len(eval_df)

In [ ]:
eval_df['predict'] = eval_df['final_text'].apply(lambda x: m.predict([x]))

eval_df.to_csv('data/city_level_bert_eval_2w_noip.csv')

In [7]:

df = pd.read_csv('data/city_level_bert_eval_2w_noip.csv')

In [8]:
def calculate(df, prob):
    # df_sa.fillna('', inplace=True)
    df['label'] = df['predict'].apply(lambda x: eval(x)[0][0])
    df['prob'] = df['predict'].apply(lambda x: eval(x)[1][0])
    df['same'] = df.apply(lambda x: x['label'] == x['cityLevelTag'], axis=1)

    df['prob'] = df['prob'].astype(float)
    filtered_df = df[df['prob'] > prob]

    print(f'Probability thredshold set to {prob}, filtered {len(filtered_df)} rows out of {len(df)} rows!')

    return filtered_df


def evaluate(df):
    # 计算 False 的数量
    true_count = df['same'].value_counts()[True]

    # 计算总数
    total_count = len(df)

    # 计算 True 的占比
    percentage = (true_count / total_count) * 100
    print(f'### Prediction accuracy: {percentage.round(2)}%')

In [9]:
eval_df = calculate(df, 0.6)
evaluate(eval_df)

Probability thredshold set to 0.6, filtered 7036 rows out of 21000 rows!
### Prediction accuracy: 80.14%


In [10]:
eval_df = calculate(df, 0.92)
evaluate(eval_df)

Probability thredshold set to 0.92, filtered 2492 rows out of 21000 rows!
### Prediction accuracy: 95.39%
